### RDKit Conformer Browser
Derived from Greg Landrum's code by Malitha Humayun Kabir<br>
As a part of GSoC 2017 <br>
Project : RDKit - 3Dmol.js integration <br>
Mentors: Paul Czodrowski and Greg Landrum <br>
Date: 28th June 2017

Note: All the codes requred for conformer browser are in IPythonConsoleIntegration.py. Please feel free to import IPythonConsoleIntegration.py in your work environment as per your convenience. <br>
A copy of IPythonConsoleIntegration.py can also be found at the following link: <br> 
https://github.com/malithakabir/notebook/blob/master/IPythonConsoleIntegration.py

In [1]:
import IPythonConsoleIntegration as ipy
from rdkit import Chem
from rdkit.Chem import AllChem
from six.moves import urllib

In [2]:
# Codes from Greg Landrum
# obj ms will have no pre calculated propery
data = [('m1','COc1ccc2[nH]c([S@@+]([O-])Cc3ncc(C)c(OC)c3C)nc2c1'),
       ('m2','COc1ccc2[nH]c([S@+]([O-])Cc3ncc(C)c(OC)c3C)nc2c1'),
       ('m3','COc1ccc2[nH]c([S+]([O-])Cc3ncc(C)c(OC)c3C)nc2c1'),
       ('m4','CCOc1ccc2[nH]c([S@@+]([O-])Cc3ncc(C)c(OCCC)c3C)nc2c1')]
ms = [(x,Chem.AddHs(Chem.MolFromSmiles(y))) for x,y in data]
params = AllChem.ETKDG()
params.numThreads=3
for nm,m in ms:
    AllChem.EmbedMultipleConfs(m,numConfs=10,params=params)
    # align to one of the ring systems:
    AllChem.AlignMolConformers(m,m.GetSubstructMatch(Chem.MolFromSmarts('c1[nH]c2ccccc2n1')))

In [3]:
# Codes from Paolo Tosco (Slightly modified version)
# obj ms2 will have some pre calculated properties
url = 'https://github.com/rdkit/rdkit/raw/master/Docs/Book/data/cdk2.sdf'
response = urllib.request.urlopen(url)
data = response.read()
suppl = Chem.SDMolSupplier()
suppl.SetData(data)
ms2=[mol for mol in suppl]
params = AllChem.ETKDG()
params.numThreads=3
for m in ms2:
    AllChem.EmbedMultipleConfs(m,numConfs=10,params=params)

# For RDKit Mol object with NO existing properties

In [4]:
view=ipy.startViewer(bgcolor='black')
view.show()

In [5]:
ipy.browseMolConformers(ms,view,confId=(0, 9),useDrawAs=True,drawAs='stick')

<function IPythonConsoleIntegration.addMolToViewForScrolling>

In [6]:
view.zoomTo()
view.update()

In [7]:
# Changing background color is possible too
view.setBackgroundColor('0xeeeeee')
view.update()

# For RDKit Mol object with existing properties

In [8]:
view2=ipy.startViewer(bgcolor='black')
view2.show()

In [9]:
ipy.browseMolConformers(ms2,view2,confId=(0, 9),useDrawAs=True,drawAs='stick')

<function IPythonConsoleIntegration.addMolToViewForScrolling>

In [10]:
view2.zoomTo()
view2.update()

In [11]:
# Changing background color is possible too
view2.setBackgroundColor('0xeeeeee')
view2.update()